In [1]:
#from textblob import TextBlob
#from attributegetter import *
from generatengrams import ngrammatch
from Contexts import *
import json
from Intents import *
import random
import os
import re
import datetime
from textblob import TextBlob

In [2]:
def getdate(uinput,context,attributes,matches):

    uinput = uinput.lower()
    no_patterns = 0

    monthlist = ['jan','january','feb','feburary','mar','march','apr','april','may','may','jun','june','jul','july','aug','august','sep','september','oct','october','nov','november','dec','december']
    weekdaylist = ['monday','mon','tuesday','tue','wednesday','wed','thursday','thu','friday','fri','saturday','sat','sunday','sun']


    nexttodate_pattern = '((r)(d)|(t)(h)|(n)(d))?'
    of_pattern = '((\s+)(o)(f))?'

    date_pattern1 = '([0-9]+)'
    month_pattern1 = '((\s+)(?:january|jan|feburary|feb|march|mar|april|apr|may|may|june|jun|july|jul|august|aug|september|sep|october|oct|november|nov|december|dec))'
    pattern1 = re.compile(date_pattern1+nexttodate_pattern+of_pattern+month_pattern1)
    no_patterns = no_patterns + 1

    date_pattern2 = '(([0-9])+(/|-))'
    month_pattern2 = '(([0-9])+(/|-))'
    year_pattern2 = '([0-9]+)'
    pattern2 = re.compile(date_pattern2+month_pattern2+year_pattern2)
    no_patterns = no_patterns + 1

    date_pattern3 = '((\s+)[0-9]+)'    
    month_pattern3 = '(?:jan|january|feb|feburary|mar|march|apr|april|may|may|jun|june|jul|july|aug|august|sep|september|oct|october|nov|november|dec|december)'
    pattern3 = re.compile(month_pattern3+ date_pattern3+nexttodate_pattern)
    no_patterns = no_patterns + 1

    pattern4 = re.compile('(?:day after tomorrow|tomorrow)')

    next_pattern = '(next)'
    next_week_month_pattern = '(?:week|month)'
    pattern5 = re.compile(next_pattern+'(\s+)'+next_week_month_pattern)


    date_pattern6 = '([0-9]+)'
    next_month_pattern = '((\s+)(next month))'
    pattern6 = re.compile(date_pattern6+nexttodate_pattern+of_pattern+next_month_pattern)


    pattern7 = re.compile('(today)')

    weekday_pattern = '(?:monday|mon|tuesday|tue|wednesday|wed|thursday|thu|friday|fri|saturday|sat|sunday|sun)'
    next_week_pattern = '((\s+)(next week))'
    pattern8 = re.compile(weekday_pattern+of_pattern+next_week_pattern)

    date = re.search(pattern8,uinput)
    if(date!=None):
        matches['Date'].append(date.group())
        weekday = re.search(weekday_pattern,date.group())
        weekday = weekdaylist.index(weekday.group())/2 + 1
        deltadays = 6 + (weekday-datetime.date.today().weekday())
        date = datetime.date.today() + datetime.timedelta(deltadays)
        attributes['Date'] = date
        return attributes,matches

    date = re.search(pattern7,uinput)
    if(date!=None):
        matches['Date'].append(date.group())
        date = datetime.date.today()
        attributes['Date'] = date
        return attributes,matches

    date = re.search(pattern6,uinput)
    if(date!=None):
        matches['Date'].append(date.group())
        day = re.search(date_pattern6,date.group()).group()
        date = datetime.date(year=datetime.date.today().year,day=int(day),month=datetime.date.today().month + 1)
        attributes['Date'] = date
        return attributes,matches

    date = re.search(pattern5,uinput)
    if(date!=None):
        matches['Date'].append(date.group())

        if(date.group()=="next week"):
            date = datetime.date.today()
            date = date + datetime.timedelta(7-date.weekday())

        else:
            date = datetime.date(year=datetime.date.today().year,day=1,month=datetime.date.today().month + 1)
        attributes['Date'] = date
        return attributes,matches

    date = re.search(pattern4,uinput)
    if(date!=None):
        matches['Date'].append(date.group())
        if(date.group()=="day after tomorrow"):
            date = datetime.date.today() + datetime.timedelta(days=2)
        else:
            date = datetime.date.today() + datetime.timedelta(days=1)
        attributes['Date'] = date
        return attributes,matches

    date = re.search(pattern2,uinput)
    if(date!=None):
        matches['Date'].append(date.group())
        date = date.group()
        year = date.split('/')[2]
        day = date.split('/')[0]
        month = date.split('/')[1]
        try:
            date = datetime.date(year=int(year),month=int(month),day=int(day))
            attributes['Date'] = date
            return attributes,matches
        except ValueError as e:
            print (e)
            return attributes,matches


    date = re.search(pattern1,uinput)
    if(date!=None):
        matches['Date'].append(date.group())

        date = date.group()
        day = re.search(date_pattern1,date)
        month = re.search(month_pattern1,date)
        year = datetime.date.today().year
        month = monthlist.index(month.group().strip())/2+ 1
        try:
            date = datetime.date(year=year,month=month,day=int(day.group()))
            attributes['Date'] = date
            return attributes,matches
        except ValueError as e:
            print (e)
            return attributes,matches

    date = re.search(pattern3,uinput)
    if(date!=None):
        matches['Date'].append(date.group())

        date = date.group()
        day = re.search(date_pattern3,date)
        month = re.search(month_pattern3,date)
        year = datetime.date.today().year

        month = monthlist.index(month.group().strip())/2+ 1
        try:
            date = datetime.date(year=year,month=month,day=int(day.group()))
            attributes['Date'] = date
            return attributes,matches
        except ValueError as e:
            print(e)
            return attributes,matches

    return attributes,matches


def loadData(path, fileName):
    if (os.path.exists(path + '/' + fileName) == False):
        return None, None

    data = []
    with open(path + '/' + fileName) as f:
        lines = f.readlines()
        
        for i,line in enumerate(lines):
            data.append(line[:-1])
    return None, data

def initialGreeting():
    print ('BOT: Hi! How may I assist you?')
    print ("BOT: Please enter Done or exit to stop")

In [3]:
def check_actions(current_intent, attributes, context):
    '''This function performs the action for the intent
    as mentioned in the intent config file'''
    '''Performs actions pertaining to current intent
    for action in current_intent.actions:
        if action.contexts_satisfied(active_contexts):
            return perform_action()
    '''

    context = IntentComplete()
    return 'action: ' + current_intent.action, context

def check_required_params(current_intent, attributes, context, user_input):
    '''Collects attributes pertaining to the current intent'''
    
    for para in current_intent.params:
        if para.name == 'doctor' and user_input == 'yes':
            context = Doctor()
            return random.choice(para.prompts), context, para.name
        
        if para.required == 'True':
            if para.name not in attributes:
                if para.name=='RegNo':
                    context = GetRegNo()
                return random.choice(para.prompts), context, para.name

    return None, context, ''


def input_processor(user_input, context, attributes, intent):
    '''Spellcheck and entity extraction functions go here'''
    
    uinput = TextBlob(user_input).correct().string
    
    #update the attributes, abstract over the entities in user input
    attributes, cleaned_input = getattributes(user_input, context, attributes)
    
    return attributes, cleaned_input

def loadIntent(path, intent):
    with open(path) as fil:
        dat = json.load(fil)
        intent = dat[intent]
        return Intent(intent['intentname'],intent['Parameters'], intent['actions'])

def intentIdentifier(clean_input, context,current_intent):
    clean_input = clean_input.lower()
    scores = ngrammatch(clean_input)
    scores = sorted_by_second = sorted(scores, key=lambda tup: tup[1])
    # print clean_input
    #print 'scores', scores
    

    print('Intent name --- ', clean_input)
    if(current_intent==None):
        if("appointment" in clean_input):
            return loadIntent('params/newparams.cfg', 'Appointment')
        elif("restaurant" in clean_input):
            return loadIntent('params/newparams.cfg', 'BookRestaurant')
        else:
            return loadIntent('params/newparams.cfg',scores[-1][0])
    else:
        #print 'same intent'
        return current_intent

def getattributes(uinput,context,attributes):
    '''This function marks the entities in user input, and updates
    the attributes dictionary'''
    #Can use context to to context specific attribute fetching
    if context.name.startswith('IntentComplete'):
        return attributes, uinput
    else:

        files = os.listdir('./entities/')
        entities = {}
        for fil in files:
            lines = open('./entities/'+fil).readlines()
            for i, line in enumerate(lines):
                lines[i] = line[:-1]
            entities[fil[:-4]] = '|'.join(lines)

        for entity in entities:
            for i in entities[entity].split('|'):
                if i.lower() in uinput.lower():
                    attributes[entity] = i
                    
        for entity in entities:
            uinput = re.sub(entities[entity],r'$'+entity,uinput,flags=re.IGNORECASE)

        attributes,matches = getdate(uinput,context,attributes,{'Date':[]})
        if (len(matches['Date']) > 0):
            context.active = False
            attributes['DateNTime'] = matches['Date']
            
        if context.name=='GetRegNo'  and context.active:
            match = re.search('[0-9]+', uinput)
            if match:
                uinput = re.sub('[0-9]+', '$regno', uinput)
                attributes['RegNo'] = match.group()
                context.active = False

        return attributes, uinput

In [ ]:
class Session:
    def __init__(self, attributes=None, active_contexts=[FirstGreeting(), IntentComplete() ]):
        
        '''Initialise a default session'''
        
        #Contexts are flags which control dialogue flow, see Contexts.py
        self.active_contexts = active_contexts
        self.context = FirstGreeting()
        
        #Intent tracks the current state of dialogue
        #self.current_intent = First_Greeting()
        self.current_intent = None
        
        #attributes hold the information collected over the conversation
        self.attributes = {}
        
    def update_contexts(self):
        '''Not used yet, but is intended to maintain active contexts'''
        for context in self.active_contexts:
            if context.active:
                context.decrease_lifespan()

    def reply(self, user_input):
        '''Generate response to user input'''
        
        self.attributes, clean_input = input_processor(user_input, self.context, self.attributes, self.current_intent)
        
        self.current_intent = intentIdentifier(clean_input, self.context, self.current_intent)
        
        prompt, self.context,paramName = check_required_params(self.current_intent, self.attributes, self.context, user_input)
        
        _, data = loadData('./entities/' , paramName + '.dat')
        
        print(data)
        #prompt being None means all parameters satisfied, perform the intent action
        if prompt is None:
            if self.context.name!='IntentComplete':
                prompt, self.context = check_actions(self.current_intent, self.attributes, self.context)
        
        #Resets the state after the Intent is complete
        if self.context.name=='IntentComplete':
            self.attributes = {}
            self.context = FirstGreeting()
            self.current_intent = None
        
        return prompt

In [ ]:
session = Session()

initialGreeting()

while True:
    inp = input('User: ')
    if (inp.upper() in {'DONE', 'EXIT'}) :
        print('BOT: Thank you for Visiting ABC Clinics!')
        break
    
    elif (inp.upper() in {'CANCEL'}) :
        print('Your booking is aborted. How can we assit you?')
        session = Session()
        inp = input('User: ')

    print ('BOT:', session.reply(inp))

BOT: Hi! How may I assist you?
BOT: Please enter Done or exit to stop
User: appointment
Intent name ---  appointm$specialityt
['yes', 'no']
BOT: Are you looking for any specific doctor?
User: yes
Intent name ---  $yesno
['A K Bardhan', 'Aswathi Padmakumar', 'M Sitaram', 'G S Pushpa Rani', 'Manish C Varma', 'Mahesh Gopasetty', 'Abhilash', 'Ganesh K Murthy', 'Anuradha Panda', 'Bana Rupa', 'Achal Bhagat', 'Anirban Ray', 'Alok Ranjan', 'Amit Wagh', 'Nanditha Sesikeran', 'Hemanth Vudayaraju', 'Akshay Kekaputre', 'Chandrasekhar Reddy', 'Durga Padmaja', 'Javeed Syed', 'Arun Shah', 'C Padmakar', 'A C Anand', 'A K Hand']
BOT: May I know doctor name?
User: weitfuiwurehge
Intent name ---  weitfuiwurehge
None
BOT: When do you want to visit us?
User: 10/10/2018
Intent name ---  10/10/2018
None
BOT: action: Appointment
